# 4.3: Decision Tree Implementation

## Import Statements and Starter Code

In [1]:
import importlib
import numpy as np

In [2]:
header = ["level", "lang", "tweets", "phd"]
attribute_domains = {"level": ["Senior", "Mid", "Junior"], 
        "lang": ["R", "Python", "Java"],
        "tweets": ["yes", "no"], 
        "phd": ["yes", "no"]}
X_train = [
        ["Senior", "Java", "no", "no"],
        ["Senior", "Java", "no", "yes"],
        ["Mid", "Python", "no", "no"],
        ["Junior", "Python", "no", "no"],
        ["Junior", "R", "yes", "no"],
        ["Junior", "R", "yes", "yes"],
        ["Mid", "R", "yes", "yes"],
        ["Senior", "Python", "no", "no"],
        ["Senior", "R", "yes", "no"],
        ["Junior", "Python", "yes", "no"],
        ["Senior", "Python", "yes", "yes"],
        ["Mid", "Python", "no", "yes"],
        ["Mid", "Java", "yes", "no"],
        ["Junior", "Python", "no", "yes"]
    ]

y_train = ["False", "False", "True", "True", "True", "False", "True", "False", "True", "True", "True", "True", "True", "False"]

## How do We Represent Trees in Python?
1. nested data structure (e.g. lists)
2. OOP (e.g. `MyTree` class)

for our intents and purposes, we will use a nested tree implementation. 
* At elem 0, we have the "data type" (Attribute, Value, Leaf)
* At elem 1, we have the "data" (attribute name, attribute value, class label)
* At elem 2, it depends on the data type

##### Example: Tree Solution for the Interview Dataset

This is based on the decision tree that we made in Lab Task #1 of 4.2!!!

In [5]:
interview_tree_solution = ["Attribute", "level", 
                            ["Value", "Senior", 
                                ["Attribute", "tweets", 
                                    ["Value", "yes", 
                                        ["Leaf", "True", 2, 5]
                                    ],
                                    ["Value", "no", 
                                        ["Leaf", "False", 3, 5]
                                    ]
                                ]
                            ],
                            ["Value", "Mid", 
                                ["Leaf", "True", 4, 14]
                            ],
                            ["Value", "Junior", 
                                ["Attribute", "phd", 
                                    ["Value", "yes", 
                                        ["Leaf", "False", 2, 5]
                                    ],
                                    ["Value", "no", 
                                        ["Leaf", "True", 3, 5]
                                    ]
                                ]
                            ]
                        ]

In [ ]:
# HELPER FUNCTIONS

def select_attribute(instances, attributes):
    # TODO: USE ENTROPY TO COMPUTE/CHOOSE ATTRIBUTE WITH THE SMALLEST E_new
    # for now... just choose randomly
    return np.random.choice(attributes) # my code

    # OR...
    # rand_index = np.random.randint(0, len(attributes))
    # return attributes[rand_index]

In [3]:
attribute_domains = {
                        0: ["Junior", "Mid", "Senior"],
                        1: ["Java", "Python", "R"],
                        2: ["no", "yes"],
                        3: ["no", "yes"]
                    }

def tdidt(current_instances, available_attributes):
    # basic approach (uses recursion!!):
    print("available attributes:", available_attributes)
    
    # select an attribute to split on
    attribute = select_attribute(current_instances, available_attributes)
    print("splitting on attribute", attribute)
    available_attributes.remove(attribute)
    tree = ["Attribute", attribute]
    # group data by attribute domains (creates pairwise disjoint partitions)
    # for each partition, repeat unless one of the following occurs (base case)
    #    CASE 1: all class labels of the partition are the same => make a leaf node
    #    CASE 2: no more attributes to select (clash) => handle clash w/majority vote leaf node
    #    CASE 3: no more instances to partition (empty partition) => backtrack and replace attribute node with majority vote leaf node
    return None

## Some Notes on `fit()`

* Only takes `X_train` and `y_train`, does NOT take any headers
    * ... as such we need to extract the headers (e.g. `["att0", "att1", ...]`)
* Additionally you need to extract the domains

In [ ]:
def fit_starter_code():
    # would advise that X_train and y_train get stitched together
    train = [X_train[i] + [y_train[i]] for i in range(len(X_train))]
    # next make a copy of your header since tdidt will modify the list
    available_attributes = header.copy()
    tree = tdidt(train, available_attributes)
    print("tree", tree)